# Similarity Metrics and Optimization

### Learning Objectives

* Survey metrics used to quantify image similarity 
* Present registration as an optimization problem whose objective is to find the optimal spatial transform in the presence of image noise and plausible transformations
* Survey strategies that are used to avoid issues with initialization and local minima

## Overview

## Exercise 1: 

## Exercise 2: